In [ ]:
import pandas as pd
import os
import os.path as path

# # 최대 행 수를 None으로 설정하여 모든 행이 출력되도록 함
# pd.set_option('display.max_rows', None)

# # 최대 열 수를 None으로 설정하여 모든 열이 출력되도록 함
# pd.set_option('display.max_columns', None)

# # 행과 열의 출력 옵션을 원래 상태로 되돌림
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')

DATA_PATH = ".tmp/dataset"

In [ ]:
# CSV 파일 불러오기
df = pd.read_csv(path.join(DATA_PATH, "dataset_total.csv"), low_memory=False)

# 데이터프레임 확인
df.head()  # 상위 5개 행 출력

In [ ]:
df.shape

In [ ]:
# 데이터셋의 기본 정보 확인
df.info()

In [ ]:
# 각 열의 결측치 수 확인
missing_values = df.isnull().sum()
missing_values[missing_values > 0]

In [ ]:
# 데이터 타입 확인
df.dtypes

In [ ]:
# 타겟 변수의 분포 확인
target_counts = df["TARGET"].value_counts()
print(target_counts)

## Create Demo Dataset


In [ ]:
names = [
    "강민호",
    "김현빈",
    "김희교",
    "박동현",
    "박선애",
    "성원석",
    "신동훈",
    "신지민",
    "오손빈",
    "위혜인",
    "이나은",
    "이동현",
    "이승준",
    "이승훈",
    "이유리",
    "이주영",
    "이현상",
    "장성민",
    "전영욱",
    "정명훈",
    "정지영",
    "정혜란",
    "최우진",
    "최호윤",
    "피상민",
]

In [ ]:
# LOAN_COUNT가 0 이상인 데이터와 0인 데이터를 각각 필터링
loan_count_positive = df[df["LOAN_COUNT"] > 0]
loan_count_zero = df[df["LOAN_COUNT"] == 0]

In [ ]:
# TARGET의 비율에 맞춰 LOAN_COUNT가 0 이상인 데이터와 0인 데이터에서 각각 샘플링

# TARGET이 0인 경우 12개, 1인 경우 13개를 각각 샘플링 (1:1 비율로 나누어지도록)

sample_positive_0 = loan_count_positive[loan_count_positive["TARGET"] == 0].sample(
    n=12, random_state=42
)
sample_positive_1 = loan_count_positive[loan_count_positive["TARGET"] == 1].sample(
    n=13, random_state=42
)

sample_zero_0 = loan_count_zero[loan_count_zero["TARGET"] == 0].sample(
    n=12, random_state=42
)
sample_zero_1 = loan_count_zero[loan_count_zero["TARGET"] == 1].sample(
    n=13, random_state=42
)

In [ ]:
# 1:1 비율로 데이터 결합 (12개씩 0과 1에 대해서 합침)
sampled_data_positive = pd.concat([sample_positive_0, sample_positive_1]).reset_index(
    drop=True
)
sampled_data_zero = pd.concat([sample_zero_0, sample_zero_1]).reset_index(drop=True)

In [ ]:
# 각각에서 12개씩을 결합하여 총 24개의 데이터를 만들고 남은 1개는 TARGET의 0과 1 비율에 맞게 선택
sampled_data = pd.concat(
    [
        sampled_data_positive.sample(n=12, random_state=42),
        sampled_data_zero.sample(n=12, random_state=42),
    ]
).reset_index(drop=True)

# 이미 사용된 SK_ID_CURR를 제외한 남은 샘플을 선택하기 위해 필터링
used_sk_ids = sampled_data["SK_ID_CURR"].tolist()
remaining_samples = pd.concat(
    [sample_positive_0, sample_positive_1, sample_zero_0, sample_zero_1]
)
remaining_samples = remaining_samples[
    ~remaining_samples["SK_ID_CURR"].isin(used_sk_ids)
]

# 남은 1개의 샘플을 TARGET 비율에 맞게 선택 (분포에 맞추기 위해)
target_ratio = 19032 / 244280

if target_ratio > 0.5:
    additional_sample = remaining_samples[remaining_samples["TARGET"] == 1].sample(
        n=1, random_state=42
    )
else:
    additional_sample = remaining_samples[remaining_samples["TARGET"] == 0].sample(
        n=1, random_state=42
    )

In [ ]:
# 최종 샘플 데이터에 추가하여 25개로 만듦
sampled_data = pd.concat([sampled_data, additional_sample]).reset_index(drop=True)

In [ ]:
# 샘플링한 데이터의 name 컬럼의 값을 names 리스트의 이름으로 교체
sampled_data["NAME"] = names

In [ ]:
sampled_data

In [ ]:
# name 컬럼을 SK_ID_CURR 뒤로 이동
columns = list(sampled_data.columns)
# 현재 컬럼들 중 name을 제외한 컬럼들
columns.remove("NAME")
# 새로운 컬럼 순서를 정의
new_columns = columns[:1] + ["NAME"] + columns[1:]

# 데이터프레임의 컬럼 순서를 변경
sampled_data = sampled_data[new_columns]

# 결과 확인
sampled_data

In [ ]:
# 데이터프레임을 CSV 파일로 저장
sampled_data.to_csv(
    path.join(DATA_PATH, "dataset_demo.csv"), index=False, encoding="utf-8-sig"
)

# 칼럼별 고유값 추출 (원본)


In [ ]:
# 범주형 (object) 및 boolean 컬럼만 선택
categorical_columns = df.select_dtypes(include=["object", "bool"]).columns

# 각 범주형 컬럼의 고유값을 딕셔너리로 저장
unique_values_dict = {col: df[col].unique().tolist() for col in categorical_columns}

# 결과 출력
unique_values_dict

In [ ]:
# 가장 긴 리스트의 길이를 찾음
max_length = max(len(values) for values in unique_values_dict.values())

# 각 리스트를 max_length에 맞추어 패딩
padded_dict = {
    key: values + [values[-1]] * (max_length - len(values))
    for key, values in unique_values_dict.items()
}

# 데이터프레임으로 변환
df_padded = pd.DataFrame(padded_dict)

# 결과 출력
df_padded

In [ ]:
len(unique_values_dict)

In [ ]:
max_length

In [ ]:
df_padded.shape

In [ ]:
df_padded.to_csv(
    path.join(DATA_PATH, "input_example.csv"), index=False, encoding="utf-8-sig"
)

## Train Dataset

-   Total 에서 Demo dataset 제외한 데이터셋 생성


In [ ]:
import pandas as pd
import os.path as path

DATA_PATH = ".tmp/dataset"

total_df = pd.read_csv(path.join(DATA_PATH, "dataset_total.csv"), low_memory=False)
demo_df = pd.read_csv(path.join(DATA_PATH, "dataset_demo.csv"), low_memory=False)

# demo_df['SK_ID_CURR']와 같은 값을 가진 행들을 total_df에서 제외
filtered_total_df = total_df[~total_df["SK_ID_CURR"].isin(demo_df["SK_ID_CURR"])]

# 결과 확인
print(len(total_df) - len(filtered_total_df))

# 'object' 또는 'category' 타입
category_columns = filtered_total_df.select_dtypes(
    include=["object", "category"]
).columns.tolist()

# 'int64' 또는 'float64' 타입
numeric_columns = filtered_total_df.select_dtypes(include=["number"]).columns.tolist()

# 'bool' 타입
boolean_columns = filtered_total_df.select_dtypes(include=["bool"]).columns.tolist()


print("Category Columns:", category_columns)
print("Numeric Columns:", numeric_columns)
print("Boolean Columns:", boolean_columns)
filtered_total_df.to_csv(path.join(DATA_PATH, "dataset_train.csv"), index=False)